# 工具变量 (Instrumental Variables) - 处理内生性问题

## 学习目标

通过本章学习，你将能够：

1. **理解内生性问题**：掌握内生性的来源及其对因果推断的影响
2. **掌握 IV 的三个核心假设**：相关性、排他性、外生性
3. **实现 2SLS 估计**：理解两阶段最小二乘的原理和实现
4. **诊断工具变量质量**：学会检验弱工具变量和过度识别
5. **正确解释 IV 估计**：理解 LATE 与 ATE 的区别
6. **应用于业务场景**：价格弹性、广告效果、教育回报等真实案例

---

## 业务场景引入

### 场景 1：电商价格弹性估计

你是某电商平台的数据科学家，老板想知道：**降价 10% 能带来多少销量提升？**

你的第一反应可能是：收集历史数据，回归 `销量 ~ 价格`，不就得到价格弹性了吗？

**但是！**老板提醒你：
- 🔴 **旺季需求高，商家敢涨价** → 高价格对应高销量
- 🔴 **滞销商品被迫降价** → 低价格对应低销量

直接回归会得到 **正相关**，而真实的价格弹性应该是 **负相关**！这就是 **内生性** 问题。

### 场景 2：教育回报率估计

政府想知道：**多读一年书能提高多少收入？**

直接回归 `收入 ~ 教育年限` 也有问题：
- 🔴 **能力强的人更愿意读书** → 收入高是因为能力，不是教育
- 🔴 **家庭条件好的人读书多** → 收入高是因为家庭资源，不是教育

### 工具变量的思想

找一个 **"外部冲击"**，它：
1. ✅ **影响处理变量** (如：成本影响价格，距离影响教育)
2. ✅ **不直接影响结果** (除了通过处理变量)
3. ✅ **与混淆因素无关** (外生的)

这就是 **工具变量 (Instrumental Variable, IV)**！

---

## Part 1: 什么是内生性

### 1.1 内生性的定义

在回归模型中，如果解释变量 $X$ 与误差项 $\epsilon$ 相关，即：

$$
\text{Cov}(X, \epsilon) \neq 0
$$

则称 $X$ 是 **内生的 (endogenous)**。

### 1.2 内生性的来源

| 来源 | 例子 | 后果 |
|------|------|------|
| **遗漏变量** | 能力影响教育和收入，但能力不可观测 | 估计有偏 |
| **测量误差** | 价格数据记录错误 | 系数向 0 偏 |
| **反向因果** | 收入高 → 教育投资多 | 估计有偏 |
| **同时性** | 供需同时决定价格和数量 | 估计有偏 |

### 1.3 生动比喻

**比喻：药效评估**

假设你想评估 "吃药" 对 "病情好转" 的效果：

- 🔴 **问题**：病情越重的人越吃药 → 吃药的人病情反而更严重
- ✅ **解决**：找一个 "药房距离" 作为工具变量
  - 离药房近的人更可能吃药 (相关性)
  - 药房距离不影响病情本身 (排他性)
  - 药房距离与病情严重程度无关 (外生性)

通过 "药房距离" 引起的 "吃药" 变化，来估计真实的药效！

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子
np.random.seed(42)

### 1.4 模拟内生性问题

In [ ]:
def simulate_endogeneity(n=1000):
    """
    模拟内生性问题：需求冲击同时影响价格和销量
    
    真实模型:
    - 需求: D = 100 - 2*P + θ  (θ 是需求冲击)
    - 供给: S = 50 + 3*P
    - 均衡: D = S => P = 10 + 0.2*θ
    
    真实价格弹性 = -2
    """
    # 需求冲击 (不可观测的混淆因素)
    demand_shock = np.random.normal(0, 10, n)
    
    # 均衡价格 (受需求冲击影响 -> 内生)
    price = 10 + 0.2 * demand_shock + np.random.normal(0, 1, n)
    
    # 均衡销量
    quantity = 100 - 2 * price + demand_shock + np.random.normal(0, 2, n)
    
    return pd.DataFrame({
        'price': price,
        'quantity': quantity,
        'demand_shock': demand_shock  # 实际中不可观测
    })

# 生成数据
df = simulate_endogeneity()

# OLS 估计 (有偏)
X = df[['price']].values
y = df['quantity'].values
ols_model = LinearRegression().fit(X, y)
ols_coef = ols_model.coef_[0]

print(f"真实价格弹性: -2.0")
print(f"OLS 估计弹性: {ols_coef:.2f}")
print(f"\n❌ OLS 严重有偏！估计出正相关，而真实是负相关！")

In [ ]:
# 可视化内生性问题
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('OLS 估计 (有偏)', '真实关系 (控制需求冲击)')
)

# 左图：OLS 回归
fig.add_trace(
    go.Scatter(
        x=df['price'], 
        y=df['quantity'],
        mode='markers',
        marker=dict(color='lightblue', size=6, opacity=0.6),
        name='观测数据'
    ),
    row=1, col=1
)

x_range = np.linspace(df['price'].min(), df['price'].max(), 100)
y_pred = ols_model.predict(x_range.reshape(-1, 1))
fig.add_trace(
    go.Scatter(
        x=x_range,
        y=y_pred,
        mode='lines',
        line=dict(color='red', width=3),
        name=f'OLS: slope={ols_coef:.2f}'
    ),
    row=1, col=1
)

# 右图：按需求冲击分组，显示真实负相关
df['shock_group'] = pd.qcut(df['demand_shock'], q=5, labels=['很低', '低', '中', '高', '很高'])
colors = px.colors.sequential.Blues_r

for i, group in enumerate(['很低', '低', '中', '高', '很高']):
    group_data = df[df['shock_group'] == group]
    fig.add_trace(
        go.Scatter(
            x=group_data['price'],
            y=group_data['quantity'],
            mode='markers',
            marker=dict(color=colors[i], size=8, opacity=0.7),
            name=f'需求冲击: {group}'
        ),
        row=1, col=2
    )

# 添加真实斜率的参考线
for group in ['很低', '低', '中', '高', '很高']:
    group_data = df[df['shock_group'] == group]
    if len(group_data) > 0:
        # 拟合线性回归
        X_group = group_data[['price']].values
        y_group = group_data['quantity'].values
        model_group = LinearRegression().fit(X_group, y_group)
        
        x_line = np.linspace(group_data['price'].min(), group_data['price'].max(), 100)
        y_line = model_group.predict(x_line.reshape(-1, 1))
        
        fig.add_trace(
            go.Scatter(
                x=x_line,
                y=y_line,
                mode='lines',
                line=dict(color='gray', width=1, dash='dash'),
                showlegend=False
            ),
            row=1, col=2
        )

fig.update_xaxes(title_text="价格", row=1, col=1)
fig.update_xaxes(title_text="价格", row=1, col=2)
fig.update_yaxes(title_text="销量", row=1, col=1)
fig.update_yaxes(title_text="销量", row=1, col=2)

fig.update_layout(
    height=500,
    title_text="内生性问题示例：需求冲击导致 OLS 有偏",
    showlegend=True,
    template='plotly_white'
)

fig.show()

print("📊 解读：")
print("- 左图：所有数据混在一起，OLS 估计出正相关 (错误!)")
print("- 右图：控制需求冲击后，每组内都是负相关 (正确!)")
print("- 问题根源：需求冲击同时影响价格和销量，导致混淆偏差")

---

## Part 2: 工具变量的三个假设

### 2.1 符号定义

我们想估计以下因果效应：

$$
Y = \beta_0 + \beta_1 X + \epsilon
$$

其中：
- $Y$: 结果变量 (outcome, 如销量、收入)
- $X$: 处理变量 (treatment, 如价格、教育年限)
- $\epsilon$: 误差项 (包含不可观测的混淆因素)
- $Z$: 工具变量 (instrument)

### 2.2 三个核心假设

#### 假设 1: 相关性 (Relevance)

$$
\text{Cov}(Z, X) \neq 0
$$

**含义**：工具变量 $Z$ 必须与处理变量 $X$ 相关。

**可检验性**：✅ 可以通过第一阶段回归检验 (F 统计量 > 10)

**例子**：
- 成本影响价格 ✅
- 距离影响教育年限 ✅
- 天气影响广告曝光 ✅

#### 假设 2: 排他性 (Exclusion Restriction)

$$
Z \text{ 只能通过 } X \text{ 影响 } Y
$$

**含义**：工具变量只能通过处理变量间接影响结果，不能有直接路径。

**可检验性**：❌ 不可直接检验，需要经济学逻辑支撑

**例子**：
- 成本影响销量，只能通过价格 ✅
- 距离影响收入，只能通过教育 ✅ (如果距离还影响就业机会，则不满足 ❌)

#### 假设 3: 外生性 (Exogeneity)

$$
\text{Cov}(Z, \epsilon) = 0
$$

**含义**：工具变量与误差项不相关，即与不可观测的混淆因素无关。

**可检验性**：❌ 不可直接检验，需要制度背景支撑

**例子**：
- 原材料成本的外生冲击 (如自然灾害) ✅
- 出生地与学校的距离 (随机的) ✅
- 降雨影响心情，进而影响消费 (不满足) ❌

### 2.3 因果图表示

```
有效的 IV:
      
   Z (工具变量)
   |
   v
   X (处理) --> Y (结果)
   ^              ^
   |              |
   +---- U -------+
   (不可观测混淆)

要求:
1. Z -> X (相关性)
2. Z 不直接指向 Y (排他性)
3. Z 与 U 无关 (外生性)
```

### 2.4 生动比喻

**比喻：交通限行政策评估车辆对空气质量的影响**

- **问题**：车辆数 $X$ 与污染 $Y$ 都受经济发展 $U$ 影响 (内生)
- **工具变量**：限行政策 $Z$ (随机分配单双号)

检验三个假设：
1. ✅ **相关性**：限行确实减少车辆数
2. ✅ **排他性**：限行只能通过减少车辆来改善空气 (假设没有心理安慰效应)
3. ✅ **外生性**：单双号是随机的，与经济发展无关

In [ ]:
# ===================================
# TODO 1: 模拟一个好的工具变量场景 - 完整实现
# ===================================

def simulate_good_iv(n=1000):
    """
    模拟一个满足 IV 假设的场景
    
    Z (成本冲击) -> X (价格) -> Y (销量)
                      ^          ^
                      |          |
                      +--- U ----+
                    (需求冲击)
    """
    # 不可观测的需求冲击（混淆因子）
    demand_shock = np.random.normal(0, 10, n)
    
    # 创建一个外生的成本冲击 Z（与需求冲击无关）
    cost_shock = np.random.normal(0, 5, n)
    
    # 价格受成本冲击和需求冲击影响
    # 公式: P = 10 + 0.5*Z + 0.2*U + noise
    price = 10 + 0.5 * cost_shock + 0.2 * demand_shock + np.random.normal(0, 1, n)
    
    # 销量只受价格和需求冲击影响（成本不直接影响销量）
    # 公式: Q = 100 - 2*P + U + noise
    # 真实的因果效应: ∂Q/∂P = -2
    quantity = 100 - 2 * price + demand_shock + np.random.normal(0, 2, n)
    
    return pd.DataFrame({
        'cost_shock': cost_shock,
        'price': price,
        'quantity': quantity,
        'demand_shock': demand_shock
    })

# 生成数据
np.random.seed(42)
df_iv = simulate_good_iv()

print("✅ 数据生成完成")
print("数据预览:")
print(df_iv.head())

print("变量关系:")
print(f"  成本冲击 → 价格: {df_iv['cost_shock'].corr(df_iv['price']):.3f}")
print(f"  成本冲击 → 销量: {df_iv['cost_shock'].corr(df_iv['quantity']):.3f}")
print(f"  价格 → 销量: {df_iv['price'].corr(df_iv['quantity']):.3f}")
print(f"  需求冲击 → 价格: {df_iv['demand_shock'].corr(df_iv['price']):.3f}")
print(f"  需求冲击 → 销量: {df_iv['demand_shock'].corr(df_iv['quantity']):.3f}")

print("💡 IV 假设检查:")
print("  1. 相关性（Relevance）: 成本冲击 → 价格？")
print(f"     Cor(Z, X) = {df_iv['cost_shock'].corr(df_iv['price']):.3f} ✓ 强相关")

print("  2. 外生性（Exclusion）: 成本冲击只通过价格影响销量？")
print(f"     Cor(Z, Y | X) ≈ 0？无法直接检验，需要理论支持")

print("  3. 排除性（Exogeneity）: 成本冲击与需求冲击无关？")
print(f"     Cor(Z, U) = {df_iv['cost_shock'].corr(df_iv['demand_shock']):.3f} ✓ 几乎为 0")


In [ ]:
# 检验三个假设
def check_iv_assumptions(df):
    """
    检验 IV 的三个假设
    """
    print("=" * 60)
    print("工具变量假设检验")
    print("=" * 60)
    
    # 假设 1: 相关性 (可检验)
    corr_zx = df['cost_shock'].corr(df['price'])
    print(f"\n1️⃣  相关性假设: Corr(Z, X) = {corr_zx:.3f}")
    if abs(corr_zx) > 0.3:
        print("   ✅ 满足! 工具变量与处理变量强相关")
    else:
        print("   ❌ 不满足! 弱工具变量问题")
    
    # 假设 2: 排他性 (不可直接检验，但可以看 reduced form)
    corr_zy = df['cost_shock'].corr(df['quantity'])
    print(f"\n2️⃣  排他性假设: Corr(Z, Y) = {corr_zy:.3f}")
    print("   ⚠️  不可直接检验，需要理论支撑")
    print("   (如果 Z 与 Y 完全不相关，可能违反相关性假设)")
    
    # 假设 3: 外生性 (不可检验，但可以看与混淆因素的相关性)
    corr_zu = df['cost_shock'].corr(df['demand_shock'])
    print(f"\n3️⃣  外生性假设: Corr(Z, U) = {corr_zu:.3f}")
    if abs(corr_zu) < 0.1:
        print("   ✅ 满足! 工具变量与混淆因素基本无关")
    else:
        print("   ❌ 不满足! 工具变量可能不是外生的")
    print("   ⚠️  实际中 U 不可观测，无法直接检验")
    
    print("\n" + "=" * 60)

check_iv_assumptions(df_iv)

---

## Part 3: 两阶段最小二乘 (2SLS) 估计

### 3.1 2SLS 的直觉

既然 $X$ 是内生的，我们不能直接回归。2SLS 的思路是：

1. **第一阶段**：用工具变量 $Z$ 预测 $X$，得到 "外生部分" $\hat{X}$
2. **第二阶段**：用 $\hat{X}$ 去预测 $Y$，得到无偏估计

### 3.2 数学推导

#### 第一阶段 (First Stage)

$$
X = \pi_0 + \pi_1 Z + \nu
$$

得到预测值：

$$
\hat{X} = \hat{\pi}_0 + \hat{\pi}_1 Z
$$

**关键**：$\hat{X}$ 只包含由 $Z$ 引起的 $X$ 的变化，这部分是 **外生的**！

#### 第二阶段 (Second Stage)

$$
Y = \beta_0 + \beta_1 \hat{X} + \epsilon
$$

得到 2SLS 估计量：

$$
\hat{\beta}_{2SLS} = \frac{\text{Cov}(Z, Y)}{\text{Cov}(Z, X)}
$$

这也被称为 **Wald 估计量**。

### 3.3 生动比喻

**比喻：药效评估的随机分配**

- **问题**：重病患者更可能吃药 (内生)
- **解决**：
  1. 第一阶段：看 "药房距离" 如何影响 "吃药概率"
  2. 第二阶段：用 "由距离引起的吃药变化" 来估计药效

这样，我们只利用了 "外生的、随机的" 吃药变化，避免了病情严重性的混淆！

### 3.4 符号说明

| 符号 | 含义 |
|------|------|
| $Y$ | 结果变量 (销量、收入等) |
| $X$ | 内生处理变量 (价格、教育等) |
| $Z$ | 工具变量 (成本、距离等) |
| $\hat{X}$ | 第一阶段预测的 $X$ (外生部分) |
| $\pi_1$ | 第一阶段系数 (工具变量对处理的影响) |
| $\beta_1$ | 因果效应 (处理对结果的影响) |
| $\hat{\beta}_{2SLS}$ | 2SLS 估计量 |

In [ ]:
# ===================================
# TODO 2: 手动实现 2SLS 估计 - 完整实现
# ===================================

from sklearn.linear_model import LinearRegression

def two_stage_least_squares(Z, X, Y):
    """
    手动实现 2SLS
    
    参数:
        Z: 工具变量 (n,)
        X: 内生处理变量 (n,)
        Y: 结果变量 (n,)
    
    返回:
        results: 包含估计结果的字典
    """
    # 第一阶段 - 回归 X ~ Z
    first_stage = LinearRegression()
    first_stage.fit(Z.reshape(-1, 1), X)
    
    # 获取预测值 X_hat
    X_hat = first_stage.predict(Z.reshape(-1, 1))
    
    # 检查第一阶段 F 统计量
    ss_res = np.sum((X - X_hat)**2)
    ss_tot = np.sum((X - np.mean(X))**2)
    r2_first = 1 - ss_res / ss_tot
    n = len(X)
    f_stat = (r2_first / (1 - r2_first)) * (n - 2)
    
    # 第二阶段 - 回归 Y ~ X_hat
    second_stage = LinearRegression()
    second_stage.fit(X_hat.reshape(-1, 1), Y)
    
    # 获取 2SLS 估计量
    beta_2sls = second_stage.coef_[0]
    
    # 方法 2: 用 Wald 公式计算（理论上相同）
    cov_zy = np.cov(Z, Y)[0, 1]
    cov_zx = np.cov(Z, X)[0, 1]
    beta_wald = cov_zy / cov_zx
    
    # 计算标准误（需要调整，因为第一阶段有不确定性）
    # 简化版：使用第二阶段的残差估计方差
    y_pred = second_stage.predict(X_hat.reshape(-1, 1))
    residuals = Y - y_pred
    sigma2 = np.sum(residuals**2) / (n - 2)
    
    # 2SLS 方差（简化公式）
    var_x_hat = np.var(X_hat)
    se_2sls = np.sqrt(sigma2 / (n * var_x_hat))
    
    return {
        'beta_2sls': beta_2sls,
        'beta_wald': beta_wald,
        'se_2sls': se_2sls,
        'first_stage_r2': r2_first,
        'first_stage_f': f_stat,
        'first_stage': first_stage,
        'second_stage': second_stage,
        'X_hat': X_hat,
        'residuals': residuals
    }

# 应用 2SLS
results = two_stage_least_squares(
    df_iv['cost_shock'].values,
    df_iv['price'].values,
    df_iv['quantity'].values
)

print("=" * 70)
print("2SLS 估计结果")
print("=" * 70)

print("【第一阶段】价格 ~ 成本冲击")
print(f"  R²: {results['first_stage_r2']:.4f}")
print(f"  F-统计量: {results['first_stage_f']:.2f}")
if results['first_stage_f'] > 10:
    print(f"  ✓ F > 10，工具变量强")
else:
    print(f"  ⚠ F < 10，存在弱工具变量问题")

print("【第二阶段】销量 ~ 预测价格")
print(f"  真实因果效应: -2.0")
print(f"  2SLS 估计: {results['beta_2sls']:.4f}")
print(f"  Wald 估计: {results['beta_wald']:.4f}")
print(f"  标准误: {results['se_2sls']:.4f}")
print(f"  95% CI: [{results['beta_2sls'] - 1.96*results['se_2sls']:.4f}, "
      f"{results['beta_2sls'] + 1.96*results['se_2sls']:.4f}]")

# 对比 OLS（有偏）
from sklearn.linear_model import LinearRegression
ols = LinearRegression()
ols.fit(df_iv['price'].values.reshape(-1, 1), df_iv['quantity'].values)
beta_ols = ols.coef_[0]

print(f"【对比 OLS】")
print(f"  OLS 估计: {beta_ols:.4f} (有偏！)")
print(f"  2SLS 估计: {results['beta_2sls']:.4f} (无偏)")
print(f"  真实值: -2.0")
print(f"  OLS 偏差: {abs(beta_ols - (-2.0)):.4f}")
print(f"  2SLS 偏差: {abs(results['beta_2sls'] - (-2.0)):.4f}")

print("💡 解读:")
print("  • OLS 估计有偏，因为价格和需求冲击相关（内生性）")
print("  • 2SLS 使用成本冲击作为工具变量，消除了内生性")
print("  • 2SLS 估计更接近真实的因果效应")

print("=" * 70)


In [ ]:
# 可视化 2SLS 两个阶段
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('第一阶段: Z → X', '第二阶段: X̂ → Y')
)

# 第一阶段
fig.add_trace(
    go.Scatter(
        x=df_iv['cost_shock'],
        y=df_iv['price'],
        mode='markers',
        marker=dict(color='lightblue', size=6, opacity=0.6),
        name='观测数据'
    ),
    row=1, col=1
)

# 第一阶段拟合线
z_range = np.linspace(df_iv['cost_shock'].min(), df_iv['cost_shock'].max(), 100)
x_pred = results['first_stage'].predict(z_range.reshape(-1, 1))
fig.add_trace(
    go.Scatter(
        x=z_range,
        y=x_pred,
        mode='lines',
        line=dict(color='green', width=3),
        name='第一阶段拟合'
    ),
    row=1, col=1
)

# 第二阶段
fig.add_trace(
    go.Scatter(
        x=results['X_hat'],
        y=df_iv['quantity'],
        mode='markers',
        marker=dict(color='lightcoral', size=6, opacity=0.6),
        name='预测的 X'
    ),
    row=1, col=2
)

# 第二阶段拟合线
x_hat_range = np.linspace(results['X_hat'].min(), results['X_hat'].max(), 100)
y_pred = results['second_stage'].predict(x_hat_range.reshape(-1, 1))
fig.add_trace(
    go.Scatter(
        x=x_hat_range,
        y=y_pred,
        mode='lines',
        line=dict(color='red', width=3),
        name=f'第二阶段拟合 (slope={results["beta_2sls"]:.2f})'
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="成本冲击 (Z)", row=1, col=1)
fig.update_xaxes(title_text="预测的价格 (X̂)", row=1, col=2)
fig.update_yaxes(title_text="价格 (X)", row=1, col=1)
fig.update_yaxes(title_text="销量 (Y)", row=1, col=2)

fig.update_layout(
    height=500,
    title_text="2SLS 两阶段估计过程",
    showlegend=True,
    template='plotly_white'
)

fig.show()

print("📊 解读：")
print("- 第一阶段：提取价格中由成本冲击引起的外生变化")
print("- 第二阶段：用外生的价格变化估计对销量的影响")
print("- 关键：X̂ 是外生的，与需求冲击无关！")

In [ ]:
# 对比 OLS 和 2SLS
def compare_ols_2sls(df):
    """
    对比 OLS 和 2SLS 的估计结果
    """
    # OLS 估计
    ols = LinearRegression()
    ols.fit(df[['price']], df['quantity'])
    beta_ols = ols.coef_[0]
    
    # 2SLS 估计
    results_2sls = two_stage_least_squares(
        df['cost_shock'].values,
        df['price'].values,
        df['quantity'].values
    )
    beta_2sls = results_2sls['beta_2sls']
    
    # 结果对比
    comparison = pd.DataFrame({
        '估计方法': ['真实值', 'OLS (有偏)', '2SLS (无偏)'],
        '估计值': [-2.0, beta_ols, beta_2sls],
        '偏差': [0, beta_ols + 2.0, beta_2sls + 2.0]
    })
    
    return comparison

comparison = compare_ols_2sls(df_iv)
print(comparison.to_string(index=False))

---

## Part 4: 弱工具变量问题

### 4.1 什么是弱工具变量

如果工具变量 $Z$ 与处理变量 $X$ 的相关性很弱，即：

$$
\text{Cov}(Z, X) \approx 0
$$

则称 $Z$ 是 **弱工具变量 (weak instrument)**。

### 4.2 弱 IV 的问题

1. **有限样本偏差**：2SLS 估计量不再是无偏的
2. **推断失效**：标准误被低估，置信区间过窄
3. **放大内生性**：即使 $Z$ 与 $\epsilon$ 只有微小相关，也会导致大偏差

### 4.3 F 统计量检验

**经验法则**：第一阶段 F 统计量 > 10

$$
F = \frac{(RSS_{restricted} - RSS_{unrestricted}) / q}{RSS_{unrestricted} / (n - k)}
$$

对于单个工具变量，简化为：

$$
F = \frac{\hat{\pi}_1^2}{\text{Var}(\hat{\pi}_1)}
$$

### 4.4 生动比喻

**比喻：用温度计测体重**

假设你想估计 "运动时长" 对 "体重" 的影响，但 "运动时长" 是内生的 (爱运动的人本来就瘦)。

- 你找到一个工具变量 "气温"：天热时运动时长增加
- **问题**：气温对运动时长的影响很弱 (很多人不管天气都不运动)
- **后果**：即使气温与体重有一点点直接关系 (热天食欲差)，也会导致估计严重有偏

**结论**：弱工具变量 worse than no instrument!

In [ ]:
# 模拟弱工具变量问题
def simulate_weak_iv(n=1000, iv_strength=0.1):
    """
    模拟弱工具变量场景
    
    参数:
        iv_strength: 工具变量强度 (0.1 = 弱, 0.5 = 强)
    """
    demand_shock = np.random.normal(0, 10, n)
    cost_shock = np.random.normal(0, 5, n)
    
    # 价格只受到很小的成本冲击影响 (弱 IV)
    price = 10 + iv_strength * cost_shock + 0.5 * demand_shock + np.random.normal(0, 1, n)
    
    # 销量
    quantity = 100 - 2 * price + demand_shock + np.random.normal(0, 2, n)
    
    return pd.DataFrame({
        'cost_shock': cost_shock,
        'price': price,
        'quantity': quantity
    })

# 比较不同 IV 强度
strengths = [0.05, 0.1, 0.3, 0.5, 1.0]
results_by_strength = []

for strength in strengths:
    df_test = simulate_weak_iv(n=500, iv_strength=strength)
    
    # 第一阶段回归
    first_stage = LinearRegression()
    first_stage.fit(df_test[['cost_shock']], df_test['price'])
    
    # 计算 F 统计量
    y_pred = first_stage.predict(df_test[['cost_shock']])
    y_true = df_test['price'].values
    
    # 残差平方和
    rss = np.sum((y_true - y_pred) ** 2)
    tss = np.sum((y_true - y_true.mean()) ** 2)
    r2 = 1 - rss / tss
    
    n = len(df_test)
    k = 1  # 一个工具变量
    f_stat = (r2 / k) / ((1 - r2) / (n - k - 1))
    
    # 2SLS 估计
    results_2sls = two_stage_least_squares(
        df_test['cost_shock'].values,
        df_test['price'].values,
        df_test['quantity'].values
    )
    
    results_by_strength.append({
        'IV强度': strength,
        'F统计量': f_stat,
        '2SLS估计': results_2sls['beta_2sls'],
        '偏差': abs(results_2sls['beta_2sls'] + 2.0)
    })

df_results = pd.DataFrame(results_by_strength)
print(df_results.to_string(index=False))

print("\n📊 解读：")
print("- F < 10: 弱工具变量，估计有偏且不稳定")
print("- F > 10: 强工具变量，估计接近真实值")
print("- 经验法则：第一阶段 F 统计量至少要大于 10")

In [ ]:
# 可视化弱 IV 问题
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_results['F统计量'],
    y=df_results['2SLS估计'],
    mode='markers+lines',
    marker=dict(size=12, color=df_results['F统计量'], colorscale='Viridis', showscale=True),
    line=dict(width=2, color='gray', dash='dash'),
    text=[f"IV强度: {s}" for s in df_results['IV强度']],
    name='2SLS 估计'
))

# 真实值参考线
fig.add_hline(y=-2.0, line_dash="dash", line_color="red", 
              annotation_text="真实值 = -2.0", annotation_position="right")

# F=10 参考线
fig.add_vline(x=10, line_dash="dash", line_color="green",
              annotation_text="F = 10 (阈值)", annotation_position="top")

fig.update_layout(
    title="弱工具变量问题：F 统计量与估计偏差",
    xaxis_title="第一阶段 F 统计量",
    yaxis_title="2SLS 估计值",
    template='plotly_white',
    height=500
)

fig.show()

---

## Part 5: 过度识别检验 (Overidentification Test)

### 5.1 什么是过度识别

如果我们有 **多个工具变量**，即：

$$
\text{工具变量个数} > \text{内生变量个数}
$$

则模型是 **过度识别 (overidentified)** 的。

### 5.2 Hansen J 检验

**原假设** $H_0$: 所有工具变量都是有效的 (满足排他性假设)

**检验统计量**:

$$
J = n \cdot R^2_{\text{residuals}}
$$

其中 $R^2_{\text{residuals}}$ 是 2SLS 残差对所有工具变量回归的 $R^2$。

在原假设下：

$$
J \sim \chi^2(m - k)
$$

其中 $m$ 是工具变量个数，$k$ 是内生变量个数。

### 5.3 注意事项

⚠️ **J 检验的局限性**：
- 只能检验 "是否至少有一个 IV 无效"
- 无法告诉你 "哪个 IV 有问题"
- 需要至少一个 IV 是有效的 (无法检验所有 IV 都无效)

### 5.4 生动比喻

**比喻：多个证人的口供**

假设你有 3 个证人 (3 个 IV)，他们的证词应该一致 (都估计出相同的因果效应)。

- 如果证词矛盾 → 至少有一个证人说谎 (至少有一个 IV 无效)
- J 检验就是检验 "证词是否一致"
- 但它无法告诉你 "谁在说谎"

In [ ]:
# ===================================
# TODO 3: 实现 Hansen J 检验 - 完整实现
# ===================================

def hansen_j_test(Z_list, X, Y):
    """
    Hansen J 过度识别检验
    
    当工具变量个数 > 内生变量个数时，可以检验工具变量的有效性
    
    参数:
        Z_list: 多个工具变量的列表 [(n,), (n,), ...]
        X: 内生处理变量 (n,)
        Y: 结果变量 (n,)
    
    返回:
        results: 检验结果字典
    """
    n = len(Y)
    m = len(Z_list)  # 工具变量个数
    k = 1  # 内生变量个数
    
    if m <= k:
        return {
            '结论': '工具变量个数 ≤ 内生变量个数，无法进行过度识别检验'
        }
    
    # 构造工具变量矩阵 Z
    Z = np.column_stack(Z_list)
    
    # 第一阶段 - 回归 X ~ Z
    first_stage = LinearRegression()
    first_stage.fit(Z, X)
    X_hat = first_stage.predict(Z)
    
    # 第二阶段 - 回归 Y ~ X_hat
    second_stage = LinearRegression()
    second_stage.fit(X_hat.reshape(-1, 1), Y)
    
    # 计算 2SLS 残差
    residuals = Y - second_stage.predict(X_hat.reshape(-1, 1))
    
    # 回归残差对所有 IV
    residual_model = LinearRegression()
    residual_model.fit(Z, residuals)
    
    # 计算 R²
    y_pred = residual_model.predict(Z)
    ss_res = np.sum((residuals - y_pred) ** 2)
    ss_tot = np.sum((residuals - residuals.mean()) ** 2)
    r2 = 1 - ss_res / ss_tot
    
    # J 统计量
    j_stat = n * r2
    
    # 自由度
    df = m - k
    
    # p 值
    p_value = 1 - stats.chi2.cdf(j_stat, df)
    
    return {
        'J统计量': j_stat,
        '自由度': df,
        'p值': p_value,
        'R²': r2,
        '工具变量个数': m,
        '内生变量个数': k,
        '过度识别约束': df,
        '结论': '拒绝原假设，至少有一个IV无效' if p_value < 0.05 else '无法拒绝原假设，IV可能都有效'
    }

# 模拟多个工具变量的场景
def simulate_multiple_iv(n=1000):
    """模拟有两个工具变量的场景"""
    demand_shock = np.random.normal(0, 10, n)
    
    # 两个有效的工具变量（都与需求冲击无关）
    cost_shock_1 = np.random.normal(0, 5, n)  # 原材料成本
    cost_shock_2 = np.random.normal(0, 5, n)  # 运输成本
    
    # 价格受两个成本冲击影响
    price = 10 + 0.5 * cost_shock_1 + 0.3 * cost_shock_2 + 0.2 * demand_shock + np.random.normal(0, 1, n)
    
    # 销量
    quantity = 100 - 2 * price + demand_shock + np.random.normal(0, 2, n)
    
    return cost_shock_1, cost_shock_2, price, quantity

# 生成数据
np.random.seed(123)
Z1, Z2, X_multi, Y_multi = simulate_multiple_iv()

# 执行 J 检验
j_results = hansen_j_test([Z1, Z2], X_multi, Y_multi)

print("=" * 70)
print("Hansen J 过度识别检验")
print("=" * 70)

for key, value in j_results.items():
    if isinstance(value, float):
        print(f"{key}: {value:.4f}")
    else:
        print(f"{key}: {value}")

print("" + "=" * 70)

print("💡 解读:")
print(f"  • 我们有 {j_results['工具变量个数']} 个工具变量，{j_results['内生变量个数']} 个内生变量")
print(f"  • 过度识别约束: {j_results['过度识别约束']} 个")
print(f"  • 原假设: 所有工具变量都有效（满足排除性约束）")

if j_results['p值'] >= 0.05:
    print(f"  • p-value = {j_results['p值']:.4f} ≥ 0.05")
    print(f"  • ✓ 无法拒绝原假设，工具变量可能都有效")
    print(f"  • ✓ 数据与"所有IV都有效"的假设一致")
else:
    print(f"  • p-value = {j_results['p值']:.4f} < 0.05")
    print(f"  • ✗ 拒绝原假设，至少有一个IV无效")
    print(f"  • ⚠ 某些IV可能违反排除性约束（直接影响结果变量）")

print("⚠️  注意:")
print("  • J 检验只能检验"某些IV无效"，不能告诉你是哪一个")
print("  • 即使 J 检验通过，也不能证明所有IV都有效（可能所有IV都无效）")
print("  • J 检验依赖于"至少有 k 个IV是有效的"这个假设")

print("=" * 70)

# 案例：一个无效的 IV
print("【案例：添加一个无效的工具变量】")

# 添加一个无效的 IV（直接影响销量）
Z3_invalid = np.random.normal(0, 5, len(X_multi))
Y_invalid = Y_multi + 0.5 * Z3_invalid  # Z3 直接影响 Y（违反排除性）

j_results_invalid = hansen_j_test([Z1, Z2, Z3_invalid], X_multi, Y_invalid)

print("结果:")
for key, value in j_results_invalid.items():
    if isinstance(value, float):
        print(f"{key}: {value:.4f}")
    else:
        print(f"{key}: {value}")

if j_results_invalid['p值'] < 0.05:
    print("✓ J 检验成功检测出无效的工具变量！")


---

## Part 6: LATE 与外部效度

### 6.1 LATE 的定义

**局部平均处理效应 (Local Average Treatment Effect, LATE)**：

IV 估计的是 **Compliers** 的平均处理效应，即那些 "因为工具变量变化而改变处理状态" 的个体。

### 6.2 人群分类

根据个体对工具变量的反应，可以分为 4 类：

| 类型 | 定义 | Z=0时 | Z=1时 |
|------|------|--------|--------|
| **Compliers** | 服从工具变量 | 不处理 | 处理 |
| **Always-takers** | 总是处理 | 处理 | 处理 |
| **Never-takers** | 从不处理 | 不处理 | 不处理 |
| **Defiers** | 违抗工具变量 | 处理 | 不处理 |

**单调性假设 (Monotonicity)**：没有 Defiers。

### 6.3 LATE 公式

$$
\text{LATE} = \frac{E[Y|Z=1] - E[Y|Z=0]}{E[X|Z=1] - E[X|Z=0]}
$$

分子：工具变量对结果的 **Reduced Form** 效应  
分母：工具变量对处理的 **First Stage** 效应

### 6.4 LATE vs ATE

| 效应 | 估计对象 | 外部效度 |
|------|----------|----------|
| **ATE** | 全体人群的平均效应 | 高 |
| **LATE** | Compliers 的平均效应 | 低 |

**关键问题**：Compliers 是谁？他们有代表性吗？

### 6.5 生动比喻

**比喻：征兵抽签与服兵役的收入效应**

- **问题**：服兵役对收入的影响？
- **内生性**：爱国者更可能参军，也可能更努力工作
- **工具变量**：越战时期的征兵抽签 (随机)

人群分类：
- **Compliers**：抽中就去，没抽中就不去 (征兵对象)
- **Always-takers**：抽不抽中都主动参军 (志愿者)
- **Never-takers**：抽中也逃兵役 (逃兵)

**LATE 估计的是 Compliers 的效应**，不是全体人群的效应！

### 6.6 外部效度讨论

**何时 LATE ≈ ATE？**
1. Compliers 占比很大
2. 处理效应对所有人都相似 (效应同质性)

**何时 LATE 外部效度差？**
1. Compliers 是特殊群体 (如边际人群)
2. 处理效应异质性很强

In [ ]:
# 模拟 LATE vs ATE
def simulate_late_vs_ate(n=10000):
    """
    模拟 LATE 与 ATE 的区别
    
    场景: 鼓励使用健身APP对减重的效果
    - Z: 是否收到推送通知 (工具变量)
    - X: 是否使用健身APP (处理)
    - Y: 体重减少量 (结果)
    """
    # 个体特征：自律程度 (0-1)
    self_discipline = np.random.beta(2, 2, n)
    
    # 随机分配推送通知
    Z = np.random.binomial(1, 0.5, n)
    
    # 是否使用APP (受自律和推送影响)
    # - 高自律者：总是用 (Always-takers)
    # - 低自律者：从不用 (Never-takers)
    # - 中等自律者：收到推送才用 (Compliers)
    prob_use = 0.2 + 0.6 * self_discipline + 0.3 * Z * (1 - self_discipline)
    X = (np.random.random(n) < prob_use).astype(int)
    
    # 处理效应异质性：自律程度越高，APP效果越好
    treatment_effect = 5 + 10 * self_discipline
    
    # 体重减少量
    Y0 = 2 + 3 * self_discipline + np.random.normal(0, 1, n)  # 不用APP
    Y1 = Y0 + treatment_effect  # 用APP
    Y = X * Y1 + (1 - X) * Y0
    
    # 识别人群类型
    # 模拟 Z=0 和 Z=1 时的处理选择
    prob_use_z0 = 0.2 + 0.6 * self_discipline
    prob_use_z1 = 0.2 + 0.6 * self_discipline + 0.3 * (1 - self_discipline)
    
    X_z0 = (np.random.random(n) < prob_use_z0).astype(int)
    X_z1 = (np.random.random(n) < prob_use_z1).astype(int)
    
    types = []
    for i in range(n):
        if X_z1[i] == 1 and X_z0[i] == 0:
            types.append('Complier')
        elif X_z1[i] == 1 and X_z0[i] == 1:
            types.append('Always-taker')
        elif X_z1[i] == 0 and X_z0[i] == 0:
            types.append('Never-taker')
        else:
            types.append('Defier')
    
    df = pd.DataFrame({
        'Z': Z,
        'X': X,
        'Y': Y,
        'self_discipline': self_discipline,
        'treatment_effect': treatment_effect,
        'type': types
    })
    
    return df

df_late = simulate_late_vs_ate()

# 计算不同效应
# ATE (全体人群)
ate = df_late['treatment_effect'].mean()

# ATT (已处理人群)
att = df_late[df_late['X'] == 1]['treatment_effect'].mean()

# LATE (Compliers)
late_true = df_late[df_late['type'] == 'Complier']['treatment_effect'].mean()

# IV 估计的 LATE
late_iv = (
    (df_late[df_late['Z'] == 1]['Y'].mean() - df_late[df_late['Z'] == 0]['Y'].mean()) /
    (df_late[df_late['Z'] == 1]['X'].mean() - df_late[df_late['Z'] == 0]['X'].mean())
)

print("=" * 60)
print("不同处理效应的比较")
print("=" * 60)
print(f"ATE (全体人群): {ate:.2f}")
print(f"ATT (已处理人群): {att:.2f}")
print(f"LATE (Compliers, 真实): {late_true:.2f}")
print(f"LATE (IV 估计): {late_iv:.2f}")
print("=" * 60)

# 人群分布
type_counts = df_late['type'].value_counts()
print("\n人群类型分布:")
print(type_counts)
print(f"\nCompliers 占比: {type_counts.get('Complier', 0) / len(df_late) * 100:.1f}%")

In [ ]:
# 可视化 LATE vs ATE
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('人群类型分布', '处理效应异质性')
)

# 左图：人群类型
type_colors = {'Complier': '#2D9CDB', 'Always-taker': '#27AE60', 
               'Never-taker': '#EB5757', 'Defier': '#F2994A'}

for typ in type_counts.index:
    fig.add_trace(
        go.Bar(
            x=[typ],
            y=[type_counts[typ]],
            name=typ,
            marker_color=type_colors.get(typ, 'gray')
        ),
        row=1, col=1
    )

# 右图：处理效应分布
for typ in ['Complier', 'Always-taker', 'Never-taker']:
    if typ in df_late['type'].values:
        data = df_late[df_late['type'] == typ]['treatment_effect']
        fig.add_trace(
            go.Violin(
                y=data,
                name=typ,
                box_visible=True,
                meanline_visible=True,
                fillcolor=type_colors.get(typ, 'gray'),
                opacity=0.6
            ),
            row=1, col=2
        )

# 添加参考线
fig.add_hline(y=ate, line_dash="dash", line_color="red", 
              annotation_text=f"ATE = {ate:.1f}", 
              annotation_position="right", row=1, col=2)

fig.update_xaxes(title_text="人群类型", row=1, col=1)
fig.update_xaxes(title_text="人群类型", row=1, col=2)
fig.update_yaxes(title_text="人数", row=1, col=1)
fig.update_yaxes(title_text="处理效应 (减重 kg)", row=1, col=2)

fig.update_layout(
    height=500,
    title_text="LATE 的外部效度问题：IV 只估计 Compliers 的效应",
    showlegend=True,
    template='plotly_white'
)

fig.show()

print("📊 解读：")
print("- Compliers: 受推送影响而使用APP的人 (中等自律)")
print("- Always-takers: 无论如何都用APP的人 (高自律)")
print("- Never-takers: 无论如何都不用APP的人 (低自律)")
print("- IV 估计的是 Compliers 的效应，可能与 ATE 不同！")

---

## Part 7: 业务案例

### 案例 1: 电商价格弹性估计

**背景**：某电商平台想知道降价对销量的影响。

**问题**：
- 需求旺盛时，价格高且销量高
- 滞销商品被迫降价，价格低但销量低
- 直接回归会高估价格弹性（甚至得到正相关）

**工具变量**：供应商的原材料成本冲击
- ✅ 相关性：成本影响定价
- ✅ 排他性：成本不直接影响需求
- ✅ 外生性：原材料价格波动是外生的

In [ ]:
# 案例 1: 价格弹性估计
def case_study_price_elasticity():
    """
    电商价格弹性估计
    """
    np.random.seed(123)
    n = 2000
    
    # 需求冲击 (节假日、促销季等)
    demand_shock = np.random.normal(0, 20, n)
    
    # 成本冲击 (原材料价格波动)
    cost_shock = np.random.normal(0, 10, n)
    
    # 价格: 受成本和需求影响
    base_price = 100
    price = base_price + 0.8 * cost_shock + 0.4 * demand_shock + np.random.normal(0, 5, n)
    
    # 销量: 受价格(负)和需求(正)影响
    # 真实价格弹性 = -1.5
    base_quantity = 1000
    quantity = base_quantity - 1.5 * (price - base_price) + 2 * demand_shock + np.random.normal(0, 50, n)
    
    df = pd.DataFrame({
        'cost_shock': cost_shock,
        'price': price,
        'quantity': quantity,
        'demand_shock': demand_shock
    })
    
    return df

df_case1 = case_study_price_elasticity()

# OLS vs 2SLS
# OLS (有偏)
ols = LinearRegression()
ols.fit(df_case1[['price']], df_case1['quantity'])
elasticity_ols = ols.coef_[0]

# 2SLS (无偏)
results_2sls = two_stage_least_squares(
    df_case1['cost_shock'].values,
    df_case1['price'].values,
    df_case1['quantity'].values
)
elasticity_2sls = results_2sls['beta_2sls']

print("=" * 60)
print("案例 1: 电商价格弹性估计")
print("=" * 60)
print(f"真实价格弹性: -1.50")
print(f"OLS 估计: {elasticity_ols:.2f} ❌ (严重高估)")
print(f"2SLS 估计: {elasticity_2sls:.2f} ✅ (接近真实值)")
print("\n业务含义:")
print(f"- 价格每下降 1 元，销量增加 {abs(elasticity_2sls):.1f} 件")
print(f"- 降价 10%，预计销量提升 {abs(elasticity_2sls) * 10:.0f}%")
print("=" * 60)

### 案例 2: 在线广告效果评估

**背景**：评估广告曝光对购买转化的影响。

**问题**：
- 购买意愿高的用户更可能看到广告（推荐算法）
- 直接回归会高估广告效果

**工具变量**：服务器随机故障导致的曝光中断
- ✅ 相关性：故障减少曝光
- ✅ 排他性：故障不直接影响购买意愿
- ✅ 外生性：故障是随机的

In [ ]:
# 案例 2: 广告效果评估
def case_study_ad_effectiveness():
    """
    在线广告效果评估
    """
    np.random.seed(456)
    n = 5000
    
    # 用户购买意愿 (不可观测)
    purchase_intent = np.random.beta(2, 5, n)
    
    # 服务器故障 (随机)
    server_failure = np.random.binomial(1, 0.3, n)
    
    # 广告曝光：受购买意愿(推荐算法)和服务器故障影响
    prob_ad = 0.6 * purchase_intent * (1 - 0.7 * server_failure)
    ad_exposure = np.random.binomial(1, prob_ad)
    
    # 购买转化：受广告和购买意愿影响
    # 真实广告效果 = 0.15 (15个百分点)
    prob_purchase = 0.1 + 0.15 * ad_exposure + 0.4 * purchase_intent
    purchase = np.random.binomial(1, prob_purchase)
    
    df = pd.DataFrame({
        'server_failure': server_failure,
        'ad_exposure': ad_exposure,
        'purchase': purchase,
        'purchase_intent': purchase_intent
    })
    
    return df

df_case2 = case_study_ad_effectiveness()

# OLS (有偏)
ols = LinearRegression()
ols.fit(df_case2[['ad_exposure']], df_case2['purchase'])
effect_ols = ols.coef_[0]

# 2SLS (无偏)
results_2sls = two_stage_least_squares(
    df_case2['server_failure'].values,
    df_case2['ad_exposure'].values,
    df_case2['purchase'].values
)
effect_2sls = results_2sls['beta_2sls']

print("=" * 60)
print("案例 2: 在线广告效果评估")
print("=" * 60)
print(f"真实广告效果: 0.15 (提升15个百分点)")
print(f"OLS 估计: {effect_ols:.3f} ❌ (高估)")
print(f"2SLS 估计: {effect_2sls:.3f} ✅")
print("\n业务含义:")
print(f"- 广告曝光使购买率提升 {effect_2sls * 100:.1f} 个百分点")
print(f"- 如果基础转化率 10%，广告使转化率变为 {(0.1 + effect_2sls) * 100:.1f}%")
print("=" * 60)

### 案例 3: 教育回报率估计

**背景**：评估多读一年书对收入的影响。

**问题**：
- 能力强的人更愿意读书，收入也更高
- 家庭条件好的人读书多，收入也高
- 直接回归会高估教育回报率

**工具变量**：出生地到最近大学的距离
- ✅ 相关性：距离影响上大学的概率
- ✅ 排他性：距离不直接影响收入（假设没有地区工资差异）
- ✅ 外生性：出生地是外生的

**注意**：这是经典的 Card (1995) 研究。

In [ ]:
# 案例 3: 教育回报率
def case_study_education_return():
    """
    教育回报率估计 (Card, 1995)
    """
    np.random.seed(789)
    n = 3000
    
    # 能力 (不可观测)
    ability = np.random.normal(100, 15, n)
    
    # 到大学的距离 (km)
    distance = np.random.gamma(2, 20, n)
    
    # 教育年限：受能力和距离影响
    # 距离越远，读书越少
    education = 12 + 0.05 * ability - 0.03 * distance + np.random.normal(0, 2, n)
    education = np.clip(education, 8, 20)  # 8-20年
    
    # 年收入 (万元)：受教育和能力影响
    # 真实教育回报率 = 0.8 (每多读一年，收入增加0.8万)
    income = 5 + 0.8 * education + 0.2 * ability + np.random.normal(0, 3, n)
    income = np.maximum(income, 3)  # 最低3万
    
    df = pd.DataFrame({
        'distance': distance,
        'education': education,
        'income': income,
        'ability': ability
    })
    
    return df

df_case3 = case_study_education_return()

# OLS (有偏)
ols = LinearRegression()
ols.fit(df_case3[['education']], df_case3['income'])
return_ols = ols.coef_[0]

# 2SLS (无偏)
results_2sls = two_stage_least_squares(
    df_case3['distance'].values,
    df_case3['education'].values,
    df_case3['income'].values
)
return_2sls = results_2sls['beta_2sls']

print("=" * 60)
print("案例 3: 教育回报率估计 (Card, 1995)")
print("=" * 60)
print(f"真实教育回报率: 0.80 万元/年")
print(f"OLS 估计: {return_ols:.2f} 万元/年 ❌ (高估)")
print(f"2SLS 估计: {return_2sls:.2f} 万元/年 ✅")
print("\n业务含义:")
print(f"- 多读一年书，年收入增加 {return_2sls:.1f} 万元")
print(f"- 大学4年，预计终身收入增加 {return_2sls * 4 * 30:.0f} 万元 (工作30年)")
print("\n⚠️  注意: 这是 LATE，估计的是 '因距离而放弃读书的边际人群' 的回报率")
print("=" * 60)

---

## 练习与思考题

### 练习 1: 识别有效的工具变量

对于以下场景，判断提议的工具变量是否有效，并说明理由：

1. **研究目标**：吸烟对肺癌的影响  
   **工具变量**：香烟税  
   **问题**：检验三个假设

2. **研究目标**：警察数量对犯罪率的影响  
   **工具变量**：恐怖袭击后的警力增派  
   **问题**：检验三个假设

3. **研究目标**：健身房会员对健康的影响  
   **工具变量**：到健身房的距离  
   **问题**：检验三个假设

### 练习 2: 实现弱 IV 稳健推断

当遇到弱工具变量时，标准的 2SLS 推断失效。尝试实现 Anderson-Rubin 检验：

```python
def anderson_rubin_test(Z, X, Y, beta_null):
    """
    Anderson-Rubin 检验 (弱 IV 稳健)
    
    原假设: H0: beta = beta_null
    """
    # TODO: 实现 AR 检验
    pass
```

### 练习 3: 多个内生变量的 IV

考虑以下模型：

$$
Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \epsilon
$$

其中 $X_1$ 和 $X_2$ 都是内生的。你有 3 个工具变量 $Z_1, Z_2, Z_3$。

**任务**：
1. 模拟数据
2. 实现 2SLS 估计
3. 检验工具变量的有效性

### 思考题

1. **弱 IV 悖论**：为什么弱工具变量比没有工具变量更糟糕？

2. **LATE 的局限性**：在什么情况下，LATE 的外部效度最差？如何改进？

3. **工具变量的寻找**：在你的业务场景中，能找到哪些潜在的工具变量？如何检验它们的有效性？

4. **自然实验**：查找文献，找出 3 个使用自然实验作为 IV 的经典研究，总结它们的工具变量设计。

5. **因果推断方法选择**：
   - 什么时候用 IV？
   - 什么时候用 RDD？
   - 什么时候用 DID？
   - 它们有什么共同点和区别？

In [ ]:
# ===================================
# 练习空间 - 完整实现
# ===================================

print("=" * 70)
print("练习题答案")
print("=" * 70)

# ===================================
# 练习 1: 识别有效的工具变量
# ===================================

print("【练习 1: 识别有效的工具变量】")
print("-" * 70)

print("场景：评估教育回报率")
print("  Y = 工资")
print("  X = 教育年限（内生，因为能力影响教育和工资）")
print("  U = 能力（不可观测）")

print("候选工具变量:")
candidates = {
    '1. 父母教育水平': {
        'relevance': '强（父母教育影响子女教育）',
        'exclusion': '弱（父母教育可能直接影响工资，如社会资本）',
        'exogeneity': '强（与个人能力相关性较弱）',
        'valid': False
    },
    '2. 学区房价格': {
        'relevance': '强（房价高的学区教育质量好）',
        'exclusion': '弱（富裕家庭的孩子可能有其他优势）',
        'exogeneity': '中等',
        'valid': False
    },
    '3. 出生季度': {
        'relevance': '中等（影响入学年龄，进而影响教育年限）',
        'exclusion': '强（出生季度不太可能直接影响工资）',
        'exogeneity': '强（出生季度是随机的）',
        'valid': True,
        'note': 'Angrist & Krueger (1991) 的经典IV'
    },
    '4. 征兵抽签结果': {
        'relevance': '强（被征兵会中断教育）',
        'exclusion': '强（抽签结果本身不影响工资）',
        'exogeneity': '强（抽签是随机的）',
        'valid': True,
        'note': 'Angrist (1990) 的经典IV'
    }
}

for iv, props in candidates.items():
    print(f"{iv}")
    print(f"  相关性: {props['relevance']}")
    print(f"  排除性: {props['exclusion']}")
    print(f"  外生性: {props['exogeneity']}")
    if 'note' in props:
        print(f"  📚 {props['note']}")
    print(f"  ✓ 有效" if props['valid'] else "  ✗ 可能无效")

print("💡 关键结论:")
print("  • 最好的 IV: 出生季度、征兵抽签（随机性强）")
print("  • 需要警惕的 IV: 父母教育、学区房价（可能违反排除性）")
print("  • 理想的 IV 来源: 自然实验、政策冲击、随机化")

# ===================================
# 练习 2: Anderson-Rubin 检验
# ===================================

print("" + "=" * 70)
print("【练习 2: Anderson-Rubin 检验（弱工具变量稳健检验）】")
print("-" * 70)

def anderson_rubin_test(Z, X, Y, beta_0):
    """
    Anderson-Rubin 检验（在弱工具变量下仍然有效）
    
    原假设: beta = beta_0
    
    思想: 如果 beta = beta_0，那么
          Y - beta_0 * X = epsilon
          应该与 Z 无关
    """
    n = len(Y)
    
    # 计算 reduced-form 残差
    Y_tilde = Y - beta_0 * X
    
    # 回归 Y_tilde ~ Z
    Z_mat = Z.reshape(-1, 1)
    model = LinearRegression()
    model.fit(Z_mat, Y_tilde)
    
    # F 统计量
    y_pred = model.predict(Z_mat)
    ss_model = np.sum((y_pred - Y_tilde.mean())**2)
    ss_resid = np.sum((Y_tilde - y_pred)**2)
    
    df1 = 1  # Z 的个数
    df2 = n - 2
    
    f_stat = (ss_model / df1) / (ss_resid / df2)
    p_value = 1 - stats.f.cdf(f_stat, df1, df2)
    
    return {
        'F统计量': f_stat,
        'p值': p_value,
        '拒绝H0': p_value < 0.05
    }

# 测试不同的 beta_0
beta_candidates = np.linspace(-3, -1, 21)
ar_pvalues = []

for beta_0 in beta_candidates:
    ar_result = anderson_rubin_test(
        df_iv['cost_shock'].values,
        df_iv['price'].values,
        df_iv['quantity'].values,
        beta_0
    )
    ar_pvalues.append(ar_result['p值'])

# 找置信区间（p-value > 0.05 的区域）
ci_mask = np.array(ar_pvalues) > 0.05
if ci_mask.any():
    ci_lower = beta_candidates[ci_mask][0]
    ci_upper = beta_candidates[ci_mask][-1]
    print(f"Anderson-Rubin 95% 置信区间: [{ci_lower:.3f}, {ci_upper:.3f}]")
    print(f"真实值 -2.0 在置信区间内: {ci_lower <= -2.0 <= ci_upper}")
else:
    print("无法构造置信区间")

# 可视化
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=beta_candidates,
    y=ar_pvalues,
    mode='lines+markers',
    name='AR p-value',
    line=dict(color=COLORS['treated'], width=2)
))

fig.add_hline(y=0.05, line_dash="dash", line_color="red",
              annotation_text="α = 0.05")
fig.add_vline(x=-2.0, line_dash="dot", line_color="green",
              annotation_text="真实值")

fig.update_layout(
    title='Anderson-Rubin 检验: p-value vs 假设的 β',
    xaxis_title='β (价格对销量的效应)',
    yaxis_title='p-value',
    template='plotly_white',
    height=400
)

fig.show()

print("💡 AR 检验的优势:")
print("  • 在弱工具变量情况下仍然有效（不依赖渐近理论）")
print("  • 可以构造置信区间（反转检验）")
print("  • 缺点：功效较低（更保守）")

# ===================================
# 练习 3: 多个内生变量
# ===================================

print("" + "=" * 70)
print("【练习 3: 多个内生变量的 IV 估计】")
print("-" * 70)

print("场景：需求和供给的同时识别")
print("  需求方程: Q = α0 + α1*P + α2*Income + u_d")
print("  供给方程: Q = β0 + β1*P + β2*Cost + u_s")
print("  内生变量: P, Q（同时决定）")
print("  需要至少 2 个工具变量！")

# 模拟数据
np.random.seed(789)
n = 1000

# 外生变量
income = np.random.normal(50, 10, n)  # 收入（影响需求）
cost = np.random.normal(20, 5, n)     # 成本（影响供给）

# 需求和供给的冲击
u_d = np.random.normal(0, 5, n)
u_s = np.random.normal(0, 5, n)

# 联立方程求解
# 需求: Q = 100 - 2*P + 0.5*Income + u_d
# 供给: Q = 20 + 1.5*P - 0.3*Cost + u_s
# 均衡: 100 - 2*P + 0.5*Income + u_d = 20 + 1.5*P - 0.3*Cost + u_s

# 求解 P（均衡价格）
price_eq = (100 - 20 + 0.5*income + 0.3*cost + u_d - u_s) / (2 + 1.5)

# 求解 Q（均衡数量）
quantity_eq = 100 - 2*price_eq + 0.5*income + u_d

# 用 IV 估计需求方程
# 工具变量: Cost（只影响供给，不直接影响需求）
print("估计需求方程：Q ~ P + Income")
print("工具变量：Cost（移动供给曲线，识别需求曲线）")

# 2SLS for demand equation
Z_demand = cost.reshape(-1, 1)
X_demand = np.column_stack([price_eq, income])
Y_demand = quantity_eq

# 第一阶段: P ~ Cost + Income
first_stage_demand = LinearRegression()
first_stage_demand.fit(np.column_stack([cost, income]), price_eq)
P_hat = first_stage_demand.predict(np.column_stack([cost, income]))

# 第二阶段: Q ~ P_hat + Income
second_stage_demand = LinearRegression()
second_stage_demand.fit(np.column_stack([P_hat, income]), quantity_eq)

print(f"需求方程估计结果:")
print(f"  真实参数: α1 = -2.0 (价格弹性), α2 = 0.5 (收入效应)")
print(f"  IV 估计: α1 = {second_stage_demand.coef_[0]:.3f}, α2 = {second_stage_demand.coef_[1]:.3f}")

# 用 IV 估计供给方程
# 工具变量: Income（只影响需求，不直接影响供给）
print("估计供给方程：Q ~ P + Cost")
print("工具变量：Income（移动需求曲线，识别供给曲线）")

# 第一阶段: P ~ Income + Cost
first_stage_supply = LinearRegression()
first_stage_supply.fit(np.column_stack([income, cost]), price_eq)
P_hat_supply = first_stage_supply.predict(np.column_stack([income, cost]))

# 第二阶段: Q ~ P_hat + Cost
second_stage_supply = LinearRegression()
second_stage_supply.fit(np.column_stack([P_hat_supply, cost]), quantity_eq)

print(f"供给方程估计结果:")
print(f"  真实参数: β1 = 1.5 (价格弹性), β2 = -0.3 (成本效应)")
print(f"  IV 估计: β1 = {second_stage_supply.coef_[0]:.3f}, β2 = {second_stage_supply.coef_[1]:.3f}")

print("💡 同时方程的识别:")
print("  • 需求方程用供给侧的工具变量（Cost）识别")
print("  • 供给方程用需求侧的工具变量（Income）识别")
print("  • 这是经典的供需分析框架")

print("" + "=" * 70)
print("✅ 所有练习完成！")
print("=" * 70)


---

## 总结

### 核心要点

1. **内生性问题**
   - 遗漏变量、测量误差、反向因果、同时性
   - OLS 估计有偏

2. **工具变量的三个假设**
   - ✅ 相关性：可检验 (F > 10)
   - ⚠️ 排他性：不可检验，需要理论支撑
   - ⚠️ 外生性：不可检验，需要制度背景

3. **2SLS 估计**
   - 第一阶段：提取外生变化
   - 第二阶段：估计因果效应
   - Wald 估计量：$\frac{\text{Cov}(Z,Y)}{\text{Cov}(Z,X)}$

4. **弱工具变量**
   - F < 10：有限样本偏差、推断失效
   - 弱 IV worse than no IV

5. **LATE 解释**
   - IV 估计的是 Compliers 的效应
   - 外部效度问题：LATE ≠ ATE

### 方法对比

| 方法 | 适用场景 | 假设 | 估计对象 |
|------|----------|------|----------|
| **RCT** | 可以随机分配 | 无混淆 | ATE |
| **Matching** | 有丰富协变量 | 可忽略性 | ATT |
| **RDD** | 有连续分配变量 | 断点附近可比 | LATE |
| **DID** | 有时间维度 | 平行趋势 | ATT |
| **IV** | 有外生冲击 | 三个假设 | LATE |

### 实践建议

1. **寻找好的 IV**：
   - 自然实验 (政策、灾害、随机事件)
   - 制度规则 (分数线、配额、抽签)
   - 时空变异 (距离、时差、天气)

2. **检验 IV 有效性**：
   - 第一阶段 F 统计量 > 10
   - 过度识别检验 (Hansen J)
   - 安慰剂检验 (placebo test)

3. **解释 LATE**：
   - 明确 Compliers 是谁
   - 讨论外部效度
   - 与其他方法对比

4. **稳健性检验**：
   - 不同 IV 的结果是否一致
   - 弱 IV 稳健推断 (AR, LM)
   - 子样本分析

### 延伸阅读

**经典论文**：
- Angrist & Krueger (1991): Does Compulsory School Attendance Affect Schooling and Earnings?
- Card (1995): Using Geographic Variation in College Proximity to Estimate the Return to Schooling
- Angrist & Evans (1998): Children and Their Parents' Labor Supply

**教材**：
- Angrist & Pischke (2009): Mostly Harmless Econometrics
- Wooldridge (2010): Econometric Analysis of Cross Section and Panel Data

**Python 包**：
- `linearmodels`: IV regression
- `statsmodels`: IV2SLS
- `econml`: LATE estimation

---

## 恭喜！

你已经掌握了工具变量方法，这是准实验方法的重要组成部分。

下一步，我们将学习 **合成控制法 (Synthetic Control)**，处理总体单位的因果推断问题。

**记住**：好的工具变量是可遇不可求的，寻找和论证工具变量的有效性，往往比估计本身更重要！